In [1]:
import numpy as np
import copy
from scipy.signal import argrelextrema
#from skimage.feature import peak_local_max

In [2]:
# Data Abstraction [this is code-specific]
src = '/mnt/WS_PN107151/dsideriu/Research/Monte_Carlo/my_FEASST/Ar-CO2_slit_pore/Full_Run_SKX'
MC_output = 'colMat'

with open(src+'/'+MC_output,mode='r') as f:
    text_in = f.readlines()

    
for (lineID,line) in enumerate(text_in):
    if (line.split()[1] == 'lnz'):
        lnZ = float(line.split()[2])
    elif (line.split()[1] == 'beta'):
        beta = float(line.split()[2])
        T = 1./beta
    #print(line.strip('\n'))
    elif (line.split()[1] == 'macrostate(nmol)'): #macrostate data is after this
        macro_data = text_in[(lineID+1):]

        
#SEEMS LIKE THERE SHOULD BE A BETTER WAY TO DO THIS...
#  WHY DO I NEED TO CREATE PYTHON LISTS, THEN CONVERT THEM TO NumPy ARRAYS?
N = np.array( [ int(x.split()[0]) for x in macro_data ] )
lnPi = np.array( [ float(x.split()[1]) for x in macro_data ] )
#energy = np.array( [ float(x.split()[2]) for x in macro_data ] )
energy = np.array( [ float(x.split()[9])/float(x.split()[8]) for x in macro_data ] )
energy2 = np.array( [ float(x.split()[10])/float(x.split()[8]) for x in macro_data ] )

In [3]:
#Necessary Packages
#import numpy as np
#import copy
#from scipy.signal import argrelextrema

class FHMC_Analysis:
    def __init__(self,lnZ0,V,beta,Debug=False,Extrapolated=False):
        self.N = np.array([])
        self.lnPi = np.array([])
        self.energy = np.array([])
        self.energy2 = np.array([])
        self.lnZ0 = lnZ0
        self.V = V
        self.beta = beta
        self.Debug = Debug
        self.Extrapolated = Extrapolated
    def CheckTail_func(self,threshold):
        maxima = argrelextrema( self.lnPi, np.greater )
        delta = np.abs(self.lnPi[int(maxima[-1])] - self.lnPi[-1])
        if delta < threshold:
            return False
        else:
            return True    
    def reweight(self,lnZ_rw, Normalize=False, Output=False, CheckTail=False):
        # Reweighting operation
        lnPi_temp = self.lnPi + self.N*(lnZ_rw-self.lnZ0)
        # Shift relative to max value
        lnPi_temp = lnPi_temp - max(lnPi_temp)
        # Normalize, if desired
        if Normalize: lnPi_temp = lnPi_temp - np.log( np.sum( np.exp(lnPi_temp) ) )
        # Create a child object and replace the lnPi and lnZ
        child = copy.deepcopy(self) # double ensure that the child is a new object, not a pointer
        child.lnZ0 = lnZ
        child.lnPi = np.array(lnPi_temp)
        # Check the tail of the distribution
        if CheckTail:
            if not child.CheckTail_func(threshold=10.0):
                raise Exception('\nReweighted to lnZ = '+str(logZ)+'\n' \
                                'Insufficient Tail')

        return child
    def temperature_extrapolate(self,beta_extrap, order=1, Normalize=False, CheckTail=False):
        if self.Extrapolated:
            raise Exception('\nNO NO NO\nDo not extrapolate already-extrapolated lnPi!')
        mu = self.lnZ0/self.beta
        if order < 1:
            raise Exception('\nExtrapolation Order = '+str(order)+' not possible\n' \
                                'Check your input data')
        #First Order Extrapolation
        lnPi_temp = self.lnPi + (beta_extrap - self.beta)*(mu*self.N - self.energy)
        #NOTE: Inconsequential terms were skipped; they are all independent of N
        #      and so do not affect the relative values of Pi
        if order == 2:
            #Second Order Extrapolation            
            lnPi_temp = lnPi_temp + 0.50 * ((beta_extrap - self.beta)**2)*(self.energy2 - (self.energy)**2)
            #NOTE: As for first order extrap, inconsequential terms were skipped
        if order > 2:
            raise Exception('\nExtrapolation Order = '+str(order)+' not possible\n' \
                                'Check your input data')
        #Normalize, if desired
        if Normalize: lnPi_temp = lnPi_temp - np.log( np.sum( np.exp(lnPi_temp) ) )
        # Create a child object and replace the lnPi, lnZ and beta
        child = copy.deepcopy(self) # double ensure that the child is a new object, not a pointer
        child.beta = beta_extrap
        child.lnZ0 = self.lnZ0*beta_extrap/self.beta
        child.lnPi = lnPi_temp
        child.Extrapolated = True
        # Check the tail of the distribution
        if CheckTail:
            if not child.CheckTail_func(threshold=10.0):
                raise Exception('\nExtrapolated to beta = '+str(beta_extrap)+'\n' \
                                'Insufficient Tail')
        return child
    def phase_boundaries(self):
        maxima = argrelextrema( self.lnPi, np.greater, order=10 )
        if len(maxima) == 1:
            Nbound = self.N[-1]
            phases = 1
        elif ln(maxima) == 2:
            minima = argrelextrema( self.lnPi, np.less, order=10 )
            Nbound = int(minima[0])
            phases = 2
        else:
            raise Exception('\nMore than two maxima in lnPi\n'+\
                            'Current lnZ = '+str(self.lnZ))                       
        return phases, Nbound

#Function for computing ensemble properties
def ensemble_properties(self):
    Pi = np.exp( self.lnPi - max( self.lnPi ) )
    denom = np.sum( Pi )
    Navg = np.dot( self.N, Pi )/denom
    N2avg = np.dot( (self.N)**2, Pi )/denom
    Uavg = np.dot( self.energy, Pi )/denom
    U2avg = np.dot( self.energy2, Pi )/denom
    UNavg = np.dot( np.multiply(self.N,self.energy), Pi )/denom
    #Determine a better way to output all of these moments
    return Navg,N2avg,Uavg,U2avg,UNavg
    
#Function for computing phase properties
def phase_properties(self):
    # Phase-specific properties
    phases, Nbound = self.phase_boundaries()
    #Create lists for properties computation
    Navg = [ [0. for x in range(phases)] for i in range(2) ]
    Uavg = [0. for x in range(phases)]
    UNavg = [0. for x in range(phases)]
    for i in range(phases):
        #NOTE: the "numpy.multiply" function is the "Hadamard" Product,
        # e.g., element-wise multiplication of two tensors of the same rank;
        # it yields a tensor of the same rank as the input tensors
        if i == 0:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[:Nbound] ) )
            denom = np.sum( Pi[:Nbound] )
            for j in range(2):
                Navg[j][i] = np.dot( self.N[:Nbound]**(j+1), Pi[:Nbound] )/denom
            Uavg[i] = np.dot( self.energy[:Nbound], Pi[:Nbound] )/denom
            UNavg[i] = np.dot( np.multiply(self.N[:Nbound],self.energy[:Nbound]),Pi[:Nbound] )/denom
        elif i == 1:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[Nbound+1:]) )
            denom = np.sum( Pi[Nbound+1:] )
            for j in range(2):
                Navg[i] = np.dot( self.N[Nbound+1:]**(j+1), Pi[Nbound+1:] )/denom
            Uavg[i] = np.dot( self.energy[Nbound+1:], Pi[Nbound+1:] )/denom
            UNavg[i] = np.dot( np.multiply(self.N[Nbound+1:],self.energy[Nbound+1:]),Pi[Nbound+1:] )/denom            
    return Navg,Uavg,UNavg

def grand_potential(self):
    # Phase-specific properties
    phases, Nbound = self.phase_boundaries()
    # Create list
    Omega = [0. for x in range(phases)]
    for i in range(phases):
        if i == 0:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[:Nbound] ) )
            Omega[i]=( (np.log(Pi[0]) - np.sum(Pi[:Nbound]))/self.beta )
        elif i == 1:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[Nbound+1:]) )
            Omega[i]=( (np.log(Pi[0]) - np.sum(Pi[Nbound+1:]))/self.beta )
    return Omega

def pressure(self):
    return [x/self.V for x in grand_potential(self)]
    
#Bind Custom Functions to Class
FHMC_Analysis.phase_properties = phase_properties
FHMC_Analysis.ensemble_properties = ensemble_properties
FHMC_Analysis.grand_potential = grand_potential
FHMC_Analysis.pressure = pressure

In [4]:
# Instantiate the data object
confined = FHMC_Analysis(lnZ,1.,beta)
confined.N = N
confined.lnPi = lnPi
confined.energy = energy
confined.energy2 = energy2
#print(confined.beta)

#Tail checking
minima = argrelextrema( confined.lnPi, np.less, order=10 )
maxima = argrelextrema( confined.lnPi, np.greater, order=10 )
print(minima)
print(maxima)

(array([], dtype=int64),)
(array([209]),)


In [5]:
new = confined.reweight(lnZ_rw=lnZ+0.5,Normalize=True,CheckTail=True)

In [6]:
#Ensemble Average Properties
print(confined.phase_properties())
print(confined.ensemble_properties())
print(confined.grand_potential())
print(confined.pressure())

([[208.35707194108852], [43455.77875237735]], [-919.2290912219025], [-191845.60793211823])
(208.35707194108852, 43455.77875237735, -919.2290912219025, 847434.7174945663, -191845.60793211823)
[-135.3088730627116]
[-135.3088730627116]


In [7]:
#Reweighting Operation
new = confined.reweight(lnZ_rw=lnZ+0.5,Normalize=True,CheckTail=True)
minima = argrelextrema( confined.lnPi, np.less, order=10 )
maxima = argrelextrema( confined.lnPi, np.greater, order=10 )
print(maxima)
print(minima)
print(new.phase_properties())
print(new.ensemble_properties())
print(new.grand_potential())
print(new.pressure())

(array([209]),)
(array([], dtype=int64),)
([[225.02101581220725], [50661.15154353681]], [-1046.3651547103846], [-235663.6597605367])
(225.02101581220725, 50661.15154353681, -1046.3651547103846, 1096653.1388977626, -235663.6597605367)
[-225.14890112835124]
[-225.14890112835124]


In [25]:
#Temperature Extrapolation; First Order
reweighted = confined.reweight(lnZ_rw=lnZ+0.5,Normalize=True,CheckTail=True)

for i in range(5):
    print(reweighted.lnPi[i])
print()

dbeta = 0.1

lnPi_extrap = reweighted.temperature_extrapolate(beta_extrap=confined.beta+dbeta,Normalize=True,order=1)
for i in range(5):
    print(lnPi_extrap[i])
print()

test = copy.deepcopy(reweighted)
test.lnPi = lnPi_extrap
print(test.ensemble_properties())
print(test.grand_potential())

-254.45411128741011
-249.9742811199409
-246.1725004131689
-242.75792526193732
-239.61193098117994

-294.54081282393474
-290.14414628319616
-286.4216722292831
-283.082619307639
-280.00830244040947

(236.59522725662893, 55997.7574108227, -1138.4089195472638, 1297441.2459181994, -269506.4065424792)
[-257.9763783246764]


In [13]:
# Generic Ensemble Average Computer

def generic_property(self,property):
    # Phase-specific properties
    maxima = argrelextrema( self.lnPi, np.greater, order=10 )
    if len(maxima) == 1:
        Nbound = self.N[-1]
        phases = 1
    elif ln(maxima) == 2:
        minima = argrelextrema( self.lnPi, np.less, order=10 )
        Nbound = int(minima[0])
        phases = 2
    else:
        raise Exception('\nMore than two maxima in lnPi\n'+\
                        'Current lnZ = '+str(self.lnZ))
    #Create lists for properties computation
    Avg = [0. for x in range(phases)]
    for i in range(phases):
        #NOTE: the "numpy.multiply" function is the "Hadamard" Product,
        # e.g., element-wise multiplication of two tensors of the same rank;
        # it yields a tensor of the same rank as the input tensors
        if i == 0:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[:Nbound] ) )
            denom = np.sum( Pi[:Nbound] )
            Avg[i] = np.dot( property[:Nbound], Pi[:Nbound] )/denom
        elif i == 1:
            #  Shift lnPi to prevent round-off truncation in lnPi -> Pi conversion
            Pi = np.exp( self.lnPi - max( self.lnPi[Nbound+1:]) )
            denom = np.sum( Pi[Nbound+1:] )
            Avg[i] = np.dot( property[Nbound+1:], Pi[Nbound+1:] )/denom
    return Avg

confined.generic_property = generic_property

print(confined.phase_properties())

print(confined.generic_property(confined,confined.N))
print(confined.generic_property(confined,confined.N**2))
print(confined.generic_property(confined,confined.energy))
print(confined.generic_property(confined,np.multiply(confined.energy,confined.N)))
print(confined.generic_property(confined,confined.energy*confined.N))

([[208.35707194108852], [43455.77875237735]], [-919.2290912219025], [-191845.60793211823])
[208.35707194108852]
[43455.77875237735]
[-919.2290912219025]
[-191845.60793211823]
[-191845.60793211823]


In [17]:
#Temperature Extrapolation; First Order
reweighted = confined.reweight(lnZ_rw=lnZ+0.5,Normalize=True,CheckTail=True)

for i in range(5):
    print(reweighted.lnPi[i])
print()

dbeta = 0.1

lnPi_extrap = reweighted.temperature_extrapolate(beta_extrap=confined.beta+dbeta,Normalize=True,order=2)
for i in range(5):
    print(lnPi_extrap[i])
print()

test = copy.deepcopy(reweighted)
test.lnPi = lnPi_extrap
print(test.ensemble_properties())

-254.45411128741011
-249.9742811199409
-246.1725004131689
-242.75792526193732
-239.61193098117994

-295.2759919523446
-290.87487879615327
-287.14776509193666
-283.80384972202694
-280.7245006573612

(236.77325911821006, 56082.04039571205, -1139.839413281907, 1300701.6214997317, -270047.880096938)
